### D2.2: Data Analysis

In [4]:
import pandas as pd 
data = pd.read_csv("test.csv")
data.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,target
count,47494.000000,47379.000000,47383.000000,47414.000000,47565.000000,47447.000000,47551.000000,47468.000000,47528.000000,47494.000000,50000.00000
mean,30.826062,30.299584,30.576705,30.163791,29.421634,30.448964,30.149124,29.144223,29.559249,31.069398,21.30000
std,18.323820,18.667679,18.303263,18.541958,19.219590,18.384625,18.647909,19.331246,19.032847,17.831379,10.71505
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,16.000000,15.000000,14.000000,15.000000,12.000000,14.000000,14.000000,12.000000,12.000000,17.000000,12.00000
50%,28.000000,27.000000,28.000000,28.000000,27.000000,28.000000,27.000000,27.000000,28.000000,29.000000,24.50000
75%,46.000000,46.000000,45.000000,46.000000,46.000000,45.000000,46.000000,46.000000,45.000000,45.000000,31.00000
max,58.000000,58.000000,57.000000,62.000000,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,33.00000


In [ ]:
### D2.3: Data Wrangling

### D2.4: Feature Engineering